In [1]:
import os 
import pandas as pd 
os.chdir('D:\\')

In [253]:
# Extract active customers last 12 months
activ_cust = pd.read_csv('New Document3.csv')

In [2]:
# Extract all opt in customers
df_enriched = pd.read_csv('RE_ORSAY_NL_Score_Master_enriched20200225.csv',
                      sep=',',
                      dtype='str'
                      #encoding='utf-16',
                      #usecols=["SubscriberKey", "EmailOptInStatus", "Email_Confirmation_Date", "CardID"]
                     )

In [222]:
activ_cust = act_cust_2.copy()

In [223]:
# Add OptIn date
df_enriched = df_enriched.assign(date_optin = lambda d: pd.to_datetime(d.Email_Confirmation_Date).dt.date)
# Remove member without club card
df_enriched_club = df_enriched.loc[df_enriched['CardID'].notna()]
# Remove regains
#df_keys_club = df_keys_club.loc[~(df_keys_club.ScoreGroup == 'Regain')]
#print(str(df_keys_club.ScoreGroup.unique()))
# Remove non active customers

KeyboardInterrupt: 

In [254]:
selected_active_customer = pd.merge(df_enriched_club, 
                                    activ_cust, 
                                    left_on = 'SubscriberKey',
                                    right_on = 'Customer Salesforce ID',
                                    how = 'inner')

selected_active_customer =  selected_active_customer.loc[~(selected_active_customer['Turnover with VAT Euro'].isna())]


#selected_active_customer =  selected_active_customer.loc[~(selected_active_customer['Histo_Club member turnover last 12 months Euro (Measure)'].isna())]

In [245]:
selected_active_customer.shape

(2571645, 54)

In [255]:
selected_active_customer.shape

(1958222, 54)

In [37]:
# Select customers NL contacted for Spring Special
selected_spring_special = pd.read_csv('Spring_Special_2020_NL20200226.csv',decimal=',')

In [256]:
# Exclude customers that were contacted with NL for Spring special
temp = (pd
    .merge(selected_active_customer,selected_spring_special, left_on = 'SubscriberKey', right_on = 'ContactId', how='left'))
selected_customers = temp.loc[temp.Campaign_Control_Group__c.isna()]

In [257]:
selected_customers.shape

(686593, 77)

In [29]:
# Select customers contacted through other comm streams for Spring Special
cust_all_comm = pd.read_csv('Total_spring_2020_with_com_stream20200225.csv')

### NEED TO EXCLUDE OTHER CUSTOMERS 

In [258]:
class style:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'

In [259]:
def mask(dataf, country, percent_ctrl = 0.05, lower_threshold_for_ctrl=100, upper_threshold_for_ctrl=1000):
    # Initial dictionnary in absolute number of customers
    # Currently not used
#     dict_group = {'WH_Email': 500, 
#                   'WH': 10, 
#                   'WH_Push': 500, 
#                   'Email': 100, 
#                   'Email_Push': 1000, 
#                   'Push': 100}
    ctrl_size = (np.ceil(percent_ctrl*dataf.shape[0])).astype(int)
    if ctrl_size >= upper_threshold_for_ctrl: # Set control size to maximum of 1000 if we get more than that
            ctrl_size = int(1000)    
        #    print('Too many customers in control ' + str(dataf['comm_stream'].iloc[0]))
            print('Control size = ' + str(ctrl_size) + str(country))
    elif ctrl_size < lower_threshold_for_ctrl:
            ctrl_size = int(0)
#         mask_ctrl = np.zeros(dataf.shape[0]) # No controls if less than X individuals
        #    print('Not enough customers in control ' + str(dataf['comm_stream'].iloc[0]))
           # print('Only '+ str(dataf.shape[0]) + ' customers for ' +  str(dataf['comm_stream'].iloc[0]))
            print('ISSUE')
    else:        
            print('Between ' + str(lower_threshold_for_ctrl) 
              +  ' and ' + str(upper_threshold_for_ctrl) + ' customers in control ' + str(country))
            print(str(percent_ctrl*100) + '% taken as ctrl size (' + str(ctrl_size) + ' customers).'  + str(country))
        
    experiment_size = (dataf.shape[0] - ctrl_size)#.astype(int)

    mask_ctrl =  pd.Series(np.random.permutation(np.concatenate([np.ones(ctrl_size,
                                                                   #dtype=np.int64
                                                                        ),
                                                                 np.zeros(experiment_size,
                                                                   #dtype=np.int64
                                                                   )])))

    return mask_ctrl

In [260]:
DOI = selected_customers[['SubscriberKey',
                  'ContactId',
                  'CampaignId',
                  'CardID_x', 
                  'Country_x',
                  'Language_x']]
DOI = DOI.loc[~(DOI.Country_x.isna())]

DOI = DOI.sort_values(by='Country_x')
countries = DOI.Country_x.unique()
ctrl = pd.Series({})
for i in countries:
    country = i
    print(country)
    test = DOI.loc[DOI.Country_x == country]
    test_ctrl = mask(test, 
                     percent_ctrl = 0.05, 
                     lower_threshold_for_ctrl=500, 
                     upper_threshold_for_ctrl=1000,
                     country = country)
                    
    ctrl = ctrl.append(test_ctrl)

# Assign control column
DOI = DOI.assign(control = ctrl.astype(int).tolist())
DOI.loc[DOI.Country_x == 'SI'].control.sum()

AT
Between 500 and 1000 customers in control AT
5.0% taken as ctrl size (718 customers).AT
CH
ISSUE
CZ
Control size = 1000CZ
DE
Control size = 1000DE
HR
ISSUE
HU
Between 500 and 1000 customers in control HU
5.0% taken as ctrl size (987 customers).HU
MT
ISSUE
PL
Control size = 1000PL
RO
Control size = 1000RO
RS
ISSUE
SI
ISSUE
SK
Between 500 and 1000 customers in control SK
5.0% taken as ctrl size (656 customers).SK


0

In [261]:
# Assign coupon code
DOI = (DOI
#.loc[~(Country_x == 'PL')]
.assign(coupon_code_not_PL = lambda d: d['CardID_x'].astype(str) + '0021263')
#.assign(coupon_code_PL = lambda d: d['CardID_x'].astype(str) + '0021265')       
)
# Assign different ID for PL
DOI.loc[(DOI.Country_x == 'PL')] = (DOI
.loc[(DOI.Country_x == 'PL')]
.assign(coupon_code_not_PL = lambda d: d['CardID_x'].astype(str) + '0021265'))

In [262]:
DOI.columns = [c.lower() for c in DOI.columns]
DOI = DOI.rename(columns = {'cardid_x': 'card_number',
                            'country_x': 'country',
                            'language_x': 'language',
                            'campaign_id': 'campaign_id',
                            'contact_id': 'contact_id'})

In [263]:
# Define Campaign ID per country
dict_campaign_id = {'AT': '7011i000000Lm0ZAAS',
                    'CH': '7011i000000Lm0aAAC',
                    'CZ': '7011i000000Lm0bAAC',
                    'HR': '7011i000000Lm0cAAC',
                    'HU': '7011i000000Lm0dAAC',
                    'PL': '7011i000000Lm0eAAC',
                    'SI': '7011i000000Lm0fAAC',
                    'DE': '7011i000000Lm0gAAC',
                    'RO': '7011i000000Lm0hAAC',
                    'RS': '7011i000000Lm0iAAC',
                    'SK': '7011i000000Lm0jAAC'}
# Map Campaign ID
DOI = DOI.assign(campaignId = lambda d: d.country.map(dict_campaign_id))

In [264]:
# Define Campaign ID per country
dict_target_id = {'AT': '74cebf94-c230-4ae3-9f85-51dc7c6b280b',
                  'CH': '59935812-6cf3-433e-b0c5-5e6183e31e47',
                  'CZ': 'a4f09b91-eb54-4457-a268-2a4229a034e0',
                  'HR': 'eed8c754-41d1-4858-8530-6a79b356fe27',
                  'HU': '30d77959-b153-4311-91e1-a01b72d15753',
                  'PL': '5d6ed4c2-0499-4614-9786-b1ff157e69d5',
                  'SI': '41877613-e5a8-43a2-b59f-e6ec64c0c5f1',
                  'DE': '6927b017-678b-4e04-b7f8-f5434a01e676',
                  'RO': 'e8209db6-3b13-4d1d-b060-17c3cd34186a',
                  'RS': '1377c443-848b-429c-97fe-6d13a2f1d77b',
                  'SK': '58788a8d-32b1-40f9-ad45-8a0a3a44e557'}
# Map Campaign ID
DOI = DOI.assign(offerGroupId = lambda d: d.country.map(dict_target_id))

In [265]:
DOI = DOI.assign(validFrom = '2020-02-26T00:00:01.000Z')

In [266]:
DOI = (DOI
.assign(ReadyForMarketingCloud = lambda d: ~(d.control == 1))
.assign(Offeractivated = lambda d: ~(d.control == 1))
.assign(CommunicationStream = 'Email')
.rename(columns={'control': 'CampaignControlGroup',
                 'card_number': 'externalId',
                 'coupon_code_not_pl': 'offerCode'})
.drop(columns={'contactid', 'campaignid', 'language'})
)

In [237]:
DOI.drop(columns={'country'}).to_csv('export_additional_customers_spring_special_2.csv')#.externalId.astype('int64') 

In [267]:
DOI

,subscriberkey,externalId,country,CampaignControlGroup,offerCode,campaignId,offerGroupId,validFrom,ReadyForMarketingCloud,Offeractivated,CommunicationStream
1,0031i00000RxiDzAAJ,02247402484,AT,0,022474024840021263,7011i000000Lm0ZAAS,74cebf94-c230-4ae3-9f85-51dc7c6b280b,2020-02-26T00:00:01.000Z,True,True,Email
99137,0035800000yMTNJAA4,02242320053,AT,0,022423200530021263,7011i000000Lm0ZAAS,74cebf94-c230-4ae3-9f85-51dc7c6b280b,2020-02-26T00:00:01.000Z,True,True,Email
1215812,0031i00000RpGbDAAV,02247300294,AT,0,022473002940021263,7011i000000Lm0ZAAS,74cebf94-c230-4ae3-9f85-51dc7c6b280b,2020-02-26T00:00:01.000Z,True,True,Email
1803381,0031i00000J5MwvAAF,02246203484,AT,0,022462034840021263,7011i000000Lm0ZAAS,74cebf94-c230-4ae3-9f85-51dc7c6b280b,2020-02-26T00:00:01.000Z,True,True,Email
1803390,0031i00000SPIXaAAP,02247348804,AT,0,022473488040021263,7011i000000Lm0ZAAS,74cebf94-c230-4ae3-9f85-51dc7c6b280b,2020-02-26T00:00:01.000Z,True,True,Email
...,...,...,...,...,...,...,...,...,...,...,...
697743,0031i00000ISxK4AAL,09003466203,SK,0,090034662030021263,7011i000000Lm0jAAC,58788a8d-32b1-40f9-ad45-8a0a3a44e557,2020-02-26T00:00:01.000Z,True,True,Email
697742,0031i00000ISxGuAAL,09003483143,SK,0,090034831430021263,7011i000000Lm0jAAC,58788a8d-32b1-40f9-ad45-8a0a3a44e557,2020-02-26T00:00:01.000Z,True,True,Email
1901535,0031i00000Ry61KAAR,09003223121,SK,0,090032231210021263,7011i000000Lm0jAAC,58788a8d-32b1-40f9-ad45-8a0a3a44e557,2020-02-26T00:00:01.000Z,True,True,Email
93653,0035800000zpTT7AAM,09000507598,SK,0,090005075980021263,7011i000000Lm0jAAC,58788a8d-32b1-40f9-ad45-8a0a3a44e557,2020-02-26T00:00:01.000Z,True,True,Email


In [227]:
DOI

,subscriberkey,externalId,country,CampaignControlGroup,offerCode,campaignId,offerGroupId,validFrom,ReadyForMarketingCloud,Offeractivated,CommunicationStream
1646744,0031i00000SOYf5AAH,02246698612,AT,0,022466986120021263,7011i000000Lm0ZAAS,74cebf94-c230-4ae3-9f85-51dc7c6b280b,2020-02-26T00:00:01.000Z,True,True,Email
407957,0031i000003zVzWAAU,02245824846,AT,0,022458248460021263,7011i000000Lm0ZAAS,74cebf94-c230-4ae3-9f85-51dc7c6b280b,2020-02-26T00:00:01.000Z,True,True,Email
960289,0031i00000QSu30AAD,02247451021,AT,0,022474510210021263,7011i000000Lm0ZAAS,74cebf94-c230-4ae3-9f85-51dc7c6b280b,2020-02-26T00:00:01.000Z,True,True,Email
1837480,0031i00000RHFMFAA5,02247320709,AT,0,022473207090021263,7011i000000Lm0ZAAS,74cebf94-c230-4ae3-9f85-51dc7c6b280b,2020-02-26T00:00:01.000Z,True,True,Email
2369363,0031i00000MIO4WAAX,02200251168,AT,0,022002511680021263,7011i000000Lm0ZAAS,74cebf94-c230-4ae3-9f85-51dc7c6b280b,2020-02-26T00:00:01.000Z,True,True,Email
...,...,...,...,...,...,...,...,...,...,...,...
557907,0031i00000ISs16AAD,09003323488,SK,0,090033234880021263,7011i000000Lm0jAAC,58788a8d-32b1-40f9-ad45-8a0a3a44e557,2020-02-26T00:00:01.000Z,True,True,Email
109883,0031i00000StCdrAAF,09003758749,SK,0,090037587490021263,7011i000000Lm0jAAC,58788a8d-32b1-40f9-ad45-8a0a3a44e557,2020-02-26T00:00:01.000Z,True,True,Email
2496990,0031i00000J5uvZAAR,09002961469,SK,0,090029614690021263,7011i000000Lm0jAAC,58788a8d-32b1-40f9-ad45-8a0a3a44e557,2020-02-26T00:00:01.000Z,True,True,Email
557743,0031i00000SPIiOAAX,09003846125,SK,0,090038461250021263,7011i000000Lm0jAAC,58788a8d-32b1-40f9-ad45-8a0a3a44e557,2020-02-26T00:00:01.000Z,True,True,Email


In [239]:
DOI

,subscriberkey,externalId,country,CampaignControlGroup,offerCode,campaignId,offerGroupId,validFrom,ReadyForMarketingCloud,Offeractivated,CommunicationStream
1867042,0031i00000Rw0suAAB,02246878527,AT,0,022468785270021263,7011i000000Lm0ZAAS,74cebf94-c230-4ae3-9f85-51dc7c6b280b,2020-02-26T00:00:01.000Z,True,True,Email
785270,0031i000004o6QeAAI,02246399980,AT,0,022463999800021263,7011i000000Lm0ZAAS,74cebf94-c230-4ae3-9f85-51dc7c6b280b,2020-02-26T00:00:01.000Z,True,True,Email
1137926,0031i00000SsfNfAAJ,02201233064,AT,0,022012330640021263,7011i000000Lm0ZAAS,74cebf94-c230-4ae3-9f85-51dc7c6b280b,2020-02-26T00:00:01.000Z,True,True,Email
1137925,0031i00000SsfKCAAZ,02247518956,AT,0,022475189560021263,7011i000000Lm0ZAAS,74cebf94-c230-4ae3-9f85-51dc7c6b280b,2020-02-26T00:00:01.000Z,True,True,Email
1362976,0031i00000RwxjXAAR,02247332001,AT,0,022473320010021263,7011i000000Lm0ZAAS,74cebf94-c230-4ae3-9f85-51dc7c6b280b,2020-02-26T00:00:01.000Z,True,True,Email
...,...,...,...,...,...,...,...,...,...,...,...
122655,0031i00000SsWsNAAV,09003751781,SK,0,090037517810021263,7011i000000Lm0jAAC,58788a8d-32b1-40f9-ad45-8a0a3a44e557,2020-02-26T00:00:01.000Z,True,True,Email
2528939,0035800000zpUWWAA2,09000795046,SK,0,090007950460021263,7011i000000Lm0jAAC,58788a8d-32b1-40f9-ad45-8a0a3a44e557,2020-02-26T00:00:01.000Z,True,True,Email
639516,0031i00000SSBJUAA5,09003988126,SK,1,090039881260021263,7011i000000Lm0jAAC,58788a8d-32b1-40f9-ad45-8a0a3a44e557,2020-02-26T00:00:01.000Z,False,False,Email
372080,0031i00000RvLmZAAV,09003851259,SK,0,090038512590021263,7011i000000Lm0jAAC,58788a8d-32b1-40f9-ad45-8a0a3a44e557,2020-02-26T00:00:01.000Z,True,True,Email
